In [1]:

"""
Allocation Hours Paycom Data
"""
import os
import pandas as pd
import numpy as np
import re
from datetime import datetime


# workstation lookup dictionary
def read_workstation():
    os.chdir('G:\PI\HQ Projects\PP Optimization 2022')
    workstation = pd.read_excel('WorkstationHR.xlsx')
    # workstation dictionary created
    ws = workstation.groupby('Port')[['Workstation','Category']].apply(lambda x: x.set_index('Workstation').to_dict(orient='index')).to_dict()
    return ws

# last week paycom data
def read_lastweek():
    # holiday table
    os.chdir('G:\PI\HQ Projects\PP Optimization 2022')
    holiday = pd.read_excel('HolidayTable.xlsx')
    holiday['Date'] = pd.to_datetime(holiday['Date'], format='%m/%d/%Y')

    os.chdir('P:\Cost Accounting\Alloc Dept Time detail\Allocation Hours')
    files = [f for f in os.listdir('P:\Cost Accounting\Alloc Dept Time detail\Allocation Hours') if f.startswith('A')]
    files.sort()
    
    
    df = pd.DataFrame()
    days = 0
    for f in files[-7:]:
        parsed_dates = re.findall("\d+", f)
        dates = (datetime.strptime(parsed_dates[0], "%Y%m%d")).date()
        dates_week = dates.weekday()
        if (dates not in list(holiday['Date'].dt.date)) and (dates_week != 5 and dates_week != 6):
            temp_df = pd.read_excel(f)
            df = df.append(temp_df,ignore_index=True)
            days+=1
    
    # map payroll code with port name, add workstation category using dictionary
    conditions = [ df['Allocation Payroll Profile Code'] =='0U376', df['Allocation Payroll Profile Code'] =='0U377', 
               df['Allocation Payroll Profile Code'] =='0U378',df['Allocation Payroll Profile Code'] =='0U379',
               df['Allocation Payroll Profile Code'] =='0NA37']
    choices = [ "MA", "HU", "PH", "WP", "SH"]
    
    df['Port'] = np.select(conditions, choices, default=np.nan)
    df = df[df['Port']!='nan']
    df = df[['Allocation Payroll Profile Code','EE Code', 'Earn Code','Earn Hours','DATE','Allocation Department Detail Desc','Allocation Company Desc','Port']]

    # map ws dictionary to paycom dataframe
    w = read_workstation()
    df['Cost_category'] = df.apply(lambda x: w[x['Port']][x['Allocation Department Detail Desc']]['Category'], axis=1)
    
    #df_clean = df[ (df['Cost_category'] =='PDI') | (df['Cost_category'] =='PIO')]
    #display(df_clean.groupby(['Port','Cost_category','Earn Code']).sum())

    # separate pdi and pio working hours per day
    # SD1 is regular hours code for agency employees
    df_pdi = df[(df['Cost_category'] =='PDI') & (df['Earn Code'] == 'R') | (df['Cost_category'] =='PDI') & (df['Earn Code'] == 'SD1')]
    df_pio = df[(df['Cost_category'] =='PIO') & (df['Earn Code'] == 'R') | (df['Cost_category'] =='PIO') & (df['Earn Code'] == 'SD1')]
    
    # divided by 7 to get daily hours
    pdi_hours = round(df_pdi.groupby(['Port']).sum()/days,2)
    pdi_hours = pdi_hours.rename(columns={"Earn Hours": "PDIHoursperDay"}).reset_index()
    pio_hours = round(df_pio.groupby(['Port']).sum()/days,2)
    pio_hours = pio_hours.rename(columns={"Earn Hours": "PIOHoursperDay"}).reset_index()

    # total working hours added to dataframe
    hours = pdi_hours.merge(pio_hours, on="Port", how="left")
    hours['TotalHours'] = hours['PIOHoursperDay'] + hours['PDIHoursperDay']

    return hours

ws = read_workstation()
hours = read_lastweek()
hours_list = hours.values.tolist()